In [ ]:
pip install langchain-openai langchain-community faiss-cpu beautifulsoup4 requests

In [ ]:
import os
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import FAISS
from langchain.agents import Tool
from langchain.agents import AgentExecutor
from langchain.agents import create_tool_calling_agent
from langchain.tools import tool
from langchain_core.messages import HumanMessage, AIMessage
import requests
from bs4 import BeautifulSoup

# Initialize components
class AgenticRAG:
    def __init__(self):
        # Set your OpenAI API key
        os.environ["OPENAI_API_KEY"] = "your-api-key-here"

        # Initialize embedding model
        self.embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

        # Initialize LLM
        self.llm = ChatOpenAI(model="gpt-3.5-turbo")

        # Initialize vector store
        self.vector_store = None

        # Initialize memory for conversation history
        self.memory = []

        # Initialize tools (can be extended)
        self.tools = [
            Tool(
                name="web_search",
                func=self._web_search_tool,
                description="Useful when you need to search the web for current information"
            ),
            Tool(
                name="memory",
                func=self._check_memory,
                description="Useful when you need to recall previous conversations"
            )
        ]

        # Create agent
        self.agent = self._create_agent()

    def _create_agent(self):
        """Create a LangChain agent with tools"""
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a helpful AI assistant with access to web search and memory.
            Use the following tools when appropriate to answer questions accurately.
            When using web search results, always cite your sources."""),
            ("placeholder", "{chat_history}"),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ])

        agent = create_tool_calling_agent(self.llm, self.tools, prompt)
        return AgentExecutor(agent=agent, tools=self.tools, verbose=True)

    def fetch_and_index_content(self, urls):
        """Extraction Layer: Fetch and index content from URLs"""
        try:
            # Load documents
            loader = WebBaseLoader(urls)
            documents = loader.load()

            # Check for llms.txt (AI equivalent of robots.txt)
            for url in urls:
                self._check_llms_txt(url)

            # Split documents
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200
            )
            splits = text_splitter.split_documents(documents)

            # Create vector store
            self.vector_store = FAISS.from_documents(
                documents=splits,
                embedding=self.embedding_model
            )

            return True

        except Exception as e:
            print(f"Error fetching content: {str(e)}")
            return False

    def _check_llms_txt(self, url):
        """Check for llms.txt file"""
        try:
            robots_url = url.rstrip('/') + '/llms.txt'
            response = requests.get(robots_url, timeout=5)
            if response.status_code == 200:
                print(f"Found llms.txt at {robots_url}")
                print("Content:", response.text[:200] + "...")
        except:
            pass

    def query(self, user_query):
        """Process user query through the RAG pipeline"""
        # First check if we should use the vector store or web search
        if self.vector_store:
            # Use RAG pipeline
            response = self._rag_query(user_query)
        else:
            # Fall back to agent with web search
            response = self.agent.invoke({
                "input": user_query,
                "chat_history": self.memory
            })

        # Update memory
        self.memory.extend([
            HumanMessage(content=user_query),
            AIMessage(content=response if isinstance(response, str) else response.get('output', ''))
        ])

        return response

    def _rag_query(self, query):
        """Full RAG pipeline with vector search"""
        # Create retriever
        retriever = self.vector_store.as_retriever(search_kwargs={"k": 3})

        # Define prompt template
        template = """Answer the question based only on the following context:
        {context}

        Question: {question}

        If you don't know the answer, say you don't know.
        Always cite your sources using the provided metadata."""
        prompt = ChatPromptTemplate.from_template(template)

        # Create chain
        chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | self.llm
            | StrOutputParser()
        )

        return chain.invoke(query)

    @tool
    def _web_search_tool(self, query):
        """Tool for web searching (simplified)"""
        print(f"Searching the web for: {query}")
        # In a real implementation, would use SERP API or similar
        try:
            # Mock some results
            return "Web search results for: " + query + "\n1. Example result 1\n2. Example result 2"
        except Exception as e:
            return f"Error performing web search: {str(e)}"

    @tool
    def _check_memory(self):
        """Tool for checking conversation memory"""
        if not self.memory:
            return "No previous conversations found in memory."

        return "Conversation history:\n" + "\n".join(
            f"{msg.type}: {msg.content}"
            for msg in self.memory[-4:]  # Return last 4 messages
        )

# Example usage
if __name__ == "__main__":
    rag = AgenticRAG()

    # Fetch and index some content
    print("Fetching and indexing content...")
    rag.fetch_and_index_content([
        "https://en.wikipedia.org/wiki/Retrieval-augmented_generation",
        "https://en.wikipedia.org/wiki/Vector_database"
    ])

    # Ask some questions
    queries = [
        "What is RAG?",
        "How does vector search work?",
        "What are the benefits of using RAG with LLMs?"
    ]

    for query in queries:
        print(f"\nQuestion: {query}")
        response = rag.query(query)
        print(f"Answer: {response}")

    # Example using memory
    print("\nChecking memory...")
    print(rag._check_memory())